## Equity in Responsiveness: Analysis of Service Delays by Neighborhood

Are some neighborhoods systematically slower to receive service, even after accounting for service type?
We isolate government responsiveness by computing the delay relative to expected resolution time for each service type.


# Equity Analysis

Here we will see whether 311 requests are equitably solved.


In [ ]:
!pip install -r ../requirements.txt --quiet

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import pearsonr
import geopandas as gpd
import os

import warnings

warnings.filterwarnings("ignore")

# Create figures directory if it doesn't exist
os.makedirs("../figures", exist_ok=True)

sf_serv_req = gpd.read_parquet("../data/processed/serv_req_cleaned.parquet")
sf_tracts_gdf = gpd.read_file("../data/processed/sf_tracts_cleaned_2023.gpkg")

# Filter to closed requests only
closed_req = sf_serv_req[sf_serv_req["closed"].notna()].copy()

# Compute resolution time in hours
closed_req["resolution_hours"] = (
    closed_req["closed"] - closed_req["opened"]
).dt.total_seconds() / 3600.0
closed_req = closed_req[closed_req["resolution_hours"] >= 0]

print(f"Total closed requests: {len(closed_req)}")
print(f"Resolution time stats (hours):")
print(closed_req["resolution_hours"].describe())

### Step 1: Expected Resolution Time by Service Type

Calculate median resolution time for each service type to establish the baseline expectation.


In [ ]:
# Use request_type as the service classification
service_col = "Category"

print(f"Using service column: {service_col}")
print(f"Unique service types: {closed_req[service_col].nunique()}")

# Compute expected (median) resolution time by service type
expected_resolution = (
    closed_req.groupby(service_col)["resolution_hours"].median().reset_index()
)
expected_resolution.columns = [service_col, "expected_hours"]

print(f"\nExpected resolution time by service type (top 15):")
display(expected_resolution.nlargest(15, "expected_hours"))

# Store for later use
expected_dict = dict(
    zip(expected_resolution[service_col], expected_resolution["expected_hours"])
)

### Step 2: Calculate Delay Per Request

Compute actual delay as the difference between observed and expected resolution time.


In [ ]:
# Map expected resolution to each request
closed_req["expected_hours"] = closed_req[service_col].map(expected_dict)

# Drop requests with unmapped service types
closed_req_mapped = closed_req.dropna(subset=["expected_hours"]).copy()

# Compute delay (positive = slower than expected)
closed_req_mapped["delay_hours"] = (
    closed_req_mapped["resolution_hours"] - closed_req_mapped["expected_hours"]
)

print(
    f"Requests with mapped service types: {len(closed_req_mapped)} / {len(closed_req)}"
)
print(f"\nDelay statistics (hours):")
print(closed_req_mapped["delay_hours"].describe())
print(
    f"\nRequests faster than expected: {(closed_req_mapped['delay_hours'] < 0).sum()}"
)
print(f"Requests slower than expected: {(closed_req_mapped['delay_hours'] > 0).sum()}")

### Step 3: Spatial Join and Aggregate Delay by Tract

Match requests to census tracts and compute average delay.


In [ ]:
# Ensure same CRS
if closed_req_mapped.crs != sf_tracts_gdf.crs:
    closed_req_mapped = closed_req_mapped.to_crs(sf_tracts_gdf.crs)

# Spatial join: requests within tracts
req_with_tract = gpd.sjoin(
    closed_req_mapped,
    sf_tracts_gdf[["GEOID", "geometry"]],
    how="left",
    predicate="within",
)

# Handle GEOID column name (may have suffix from sjoin)
geoid_col_sjoin = "GEOID_right" if "GEOID_right" in req_with_tract.columns else "GEOID"

print(
    f"Requests matched to tracts: {req_with_tract[geoid_col_sjoin].notna().sum()} / {len(req_with_tract)}"
)

# Aggregate delay by tract
tract_delay = (
    req_with_tract.dropna(subset=[geoid_col_sjoin])
    .groupby(geoid_col_sjoin)
    .agg(
        {
            "delay_hours": ["mean", "median", "std", "count"],
            "resolution_hours": "mean",
            "expected_hours": "mean",
        }
    )
    .reset_index()
)

tract_delay.columns = [
    "GEOID",
    "avg_delay_hours",
    "median_delay_hours",
    "std_delay_hours",
    "request_count",
    "avg_resolution_hours",
    "avg_expected_hours",
]

print(f"\nTract delay statistics:")
print(
    tract_delay[["avg_delay_hours", "median_delay_hours", "request_count"]].describe()
)

# Sort by average delay
tract_delay_sorted = tract_delay.sort_values("avg_delay_hours", ascending=False)
print(f"\nTracts with LONGEST delays (top 10):")
display(
    tract_delay_sorted.head(10)[
        ["GEOID", "avg_delay_hours", "median_delay_hours", "request_count"]
    ]
)

print(f"\nTracts with SHORTEST delays (bottom 10):")
display(
    tract_delay_sorted.tail(10)[
        ["GEOID", "avg_delay_hours", "median_delay_hours", "request_count"]
    ]
)

### Step 4: Map Delay by Tract

Visualize which neighborhoods have longer-than-expected delays.


In [ ]:
# Merge delay data with tract geometries
tract_delay_map = sf_tracts_gdf.merge(tract_delay, on="GEOID", how="left")

# Create choropleth map of delay
fig, ax = plt.subplots(1, 1, figsize=(14, 12))

# Use red-to-green colormap: red = positive delay (slower), green = negative delay (faster)
# Set vmin/vmax based on actual data to capture all meaningful variation (5th to 95th percentile)
vmin = tract_delay["avg_delay_hours"].quantile(0.05)
vmax = tract_delay["avg_delay_hours"].quantile(0.95)

tract_delay_map.plot(
    column="avg_delay_hours",
    ax=ax,
    cmap="RdYlGn_r",  # Red for positive (slow), Green for negative (fast)
    edgecolor="gray",
    linewidth=0.5,
    legend=True,
    vmin=vmin,
    vmax=vmax,
    legend_kwds={
        "label": "Average Delay (hours)\n(+ = slower than expected)",
        "orientation": "horizontal",
        "shrink": 0.6,
    },
)

ax.set_title(
    "Service Request Delay by Census Tract\n(Actual Resolution - Expected Resolution)",
    fontsize=14,
    fontweight="bold",
)
ax.axis("off")
plt.tight_layout()
plt.savefig("../figures/delay_choropleth_by_tract.png", dpi=300, bbox_inches="tight")
plt.show()

print(f"Red areas = Neighborhoods with longer delays than expected (inequitable)")

print(f"Green areas = Neighborhoods with shorter delays than expected (advantaged)")
print(f"\n✓ Figure saved to ../figures/delay_choropleth_by_tract.png")

### Step 5: Correlation Analysis - Delay vs. Demographic/Economic Factors

Test whether delays correlate with income, housing costs, and racial composition.


In [ ]:
# Prepare correlation dataframe with relevant demographic variables
# List of demographic columns to check (using actual column names)
demographic_cols = [
    "median_household_income",
    "median_home_value",
    "median_rent",
    "poverty_rate",
    "gini_index",
    "pct_renter_occupied",
    "share_white",
    "share_black",
    "share_asian",
    "share_hispanic",
    "bachelors_plus_rate",
    "total_population",
]

# Get available columns
available_cols = [col for col in demographic_cols if col in sf_tracts_gdf.columns]

print(f"Available demographic variables: {available_cols}")

# Build correlation dataframe
corr_df = sf_tracts_gdf[["GEOID"] + available_cols].merge(
    tract_delay[["GEOID", "avg_delay_hours", "median_delay_hours", "request_count"]],
    on="GEOID",
    how="inner",
)

# Filter to tracts with at least 10 requests for reliability
corr_df = corr_df[corr_df["request_count"] >= 10]

print(f"\nCorrelation analysis based on {len(corr_df)} tracts with 10+ requests")

# Compute Pearson correlations
print("\n=== PEARSON CORRELATIONS WITH AVERAGE DELAY ===\n")
correlations = {}
for col in available_cols:
    if corr_df[col].notna().sum() > 5:  # Require at least 5 valid pairs
        r, p_val = pearsonr(
            corr_df[col].dropna(), corr_df.loc[corr_df[col].notna(), "avg_delay_hours"]
        )
        correlations[col] = {"r": r, "p_value": p_val}
        sig = (
            "***"
            if p_val < 0.001
            else "**" if p_val < 0.01 else "*" if p_val < 0.05 else "ns"
        )
        print(f"{col:30s}: r = {r:7.3f}, p = {p_val:.4f} {sig}")

print("\nSignificance levels: *** p<0.001, ** p<0.01, * p<0.05, ns = not significant")

In [ ]:
# Visualize key correlations
sig_cols = [col for col, stats in correlations.items() if stats["p_value"] < 0.05]

if len(sig_cols) > 0:
    n_plots = len(sig_cols)
    n_cols = 2
    n_rows = (n_plots + 1) // 2

    fig, axes = plt.subplots(n_rows, n_cols, figsize=(14, 4 * n_rows))
    axes = axes.flatten()

    for idx, col in enumerate(sig_cols):
        ax = axes[idx]
        r = correlations[col]["r"]

        # Scatter plot with regression line
        corr_df_clean = corr_df[[col, "avg_delay_hours"]].dropna()

        # Remove outliers from both axes using IQR method for robust visualization
        Q1_x = corr_df_clean[col].quantile(0.25)
        Q3_x = corr_df_clean[col].quantile(0.75)
        IQR_x = Q3_x - Q1_x
        mask_x = (corr_df_clean[col] >= Q1_x - 1.5 * IQR_x) & (
            corr_df_clean[col] <= Q3_x + 1.5 * IQR_x
        )

        Q1_y = corr_df_clean["avg_delay_hours"].quantile(0.25)
        Q3_y = corr_df_clean["avg_delay_hours"].quantile(0.75)
        IQR_y = Q3_y - Q1_y
        mask_y = (corr_df_clean["avg_delay_hours"] >= Q1_y - 1.5 * IQR_y) & (
            corr_df_clean["avg_delay_hours"] <= Q3_y + 1.5 * IQR_y
        )

        corr_df_filtered = corr_df_clean[mask_x & mask_y]

        ax.scatter(
            corr_df_filtered[col], corr_df_filtered["avg_delay_hours"], alpha=0.6, s=60
        )

        # Add trendline (using filtered data)
        z = np.polyfit(corr_df_filtered[col], corr_df_filtered["avg_delay_hours"], 1)
        p = np.poly1d(z)
        x_line = np.linspace(
            corr_df_filtered[col].min(), corr_df_filtered[col].max(), 100
        )
        ax.plot(x_line, p(x_line), "r--", linewidth=2, label=f"r = {r:.3f}")

        ax.set_xlabel(col, fontsize=11)
        ax.set_ylabel("Average Delay (hours)", fontsize=11)
        ax.set_title(f"{col} vs. Delay", fontsize=12, fontweight="bold")
        ax.legend()
        ax.grid(True, alpha=0.3)

    # Hide unused subplots
    for idx in range(len(sig_cols), len(axes)):
        axes[idx].axis("off")

    plt.tight_layout()
    plt.savefig(
        "../figures/demographic_delay_correlations.png", dpi=300, bbox_inches="tight"
    )
    plt.show()

    print(f"\nSignificant correlations found with: {', '.join(sig_cols)}")
    print(f"✓ Figure saved to ../figures/demographic_delay_correlations.png")

else:
    print("\nNo significant correlations found at p < 0.05")

## Silent Neighborhoods: Where Problems May Exist But Aren't Reported

Question: Where might problems exist but residents aren't reporting them?

We identify neighborhoods with high vulnerability indicators (poverty, renter share) but LOW 311 usage—potentially indicating barriers to complaint filing or underreporting of issues.


In [ ]:
# Step 1: Compute requests per capita
silent_df = sf_tracts_gdf[
    ["GEOID", "total_population", "poverty_rate", "pct_renter_occupied"]
].copy()

# Merge with request counts
silent_df = silent_df.merge(
    tract_delay[["GEOID", "request_count"]], on="GEOID", how="left"
)

# Fill NaN request counts with 0
silent_df["request_count"] = silent_df["request_count"].fillna(0)

# Compute requests per 1000 residents
silent_df["requests_per_1000"] = (
    silent_df["request_count"] / silent_df["total_population"]
) * 1000

print("Requests per 1000 residents - statistics:")
print(silent_df["requests_per_1000"].describe())

# Step 2: Create vulnerability score (higher = more vulnerable)
# Normalize and combine poverty rate and renter share
silent_df["vulnerability_score"] = (
    (silent_df["poverty_rate"] - silent_df["poverty_rate"].min())
    / (silent_df["poverty_rate"].max() - silent_df["poverty_rate"].min())
    + (silent_df["pct_renter_occupied"] - silent_df["pct_renter_occupied"].min())
    / (silent_df["pct_renter_occupied"].max() - silent_df["pct_renter_occupied"].min())
) / 2

print("\nVulnerability score - statistics:")
print(silent_df["vulnerability_score"].describe())

# Step 3: Identify "Silent" neighborhoods
# High vulnerability + low request rate (below 25th percentile)
req_25th = silent_df["requests_per_1000"].quantile(0.25)
vuln_75th = silent_df["vulnerability_score"].quantile(0.75)

silent_df["is_silent"] = (silent_df["vulnerability_score"] >= vuln_75th) & (
    silent_df["requests_per_1000"] <= req_25th
)

print(f"\n=== SILENT NEIGHBORHOODS ===")
print(
    f"Criteria: Vulnerability ≥ 75th percentile AND Requests per 1000 ≤ 25th percentile"
)
print(f"Vulnerability threshold: {vuln_75th:.3f}")
print(f"Request rate threshold: {req_25th:.2f} per 1000 residents")
print(f"\nTotal silent neighborhoods: {silent_df['is_silent'].sum()}")

silent_neighborhoods = silent_df[silent_df["is_silent"]].sort_values(
    "vulnerability_score", ascending=False
)
print(f"\nTop Silent Neighborhoods (high vulnerability, low reporting):")
display(
    silent_neighborhoods[
        [
            "GEOID",
            "poverty_rate",
            "pct_renter_occupied",
            "vulnerability_score",
            "requests_per_1000",
            "request_count",
            "total_population",
        ]
    ].head(15)
)

### Visualization: Silent Neighborhoods Map

Map showing which tracts are silent (high vulnerability + low reporting).


In [ ]:
# Merge silent indicators with tract geometries
silent_map = sf_tracts_gdf.merge(
    silent_df[["GEOID", "vulnerability_score", "requests_per_1000", "is_silent"]],
    on="GEOID",
    how="left",
)

# Create two subplots: vulnerability + request rate
fig, axes = plt.subplots(1, 2, figsize=(18, 8))

# Plot 1: Vulnerability Score
silent_map.plot(
    column="vulnerability_score",
    ax=axes[0],
    cmap="YlOrRd",
    edgecolor="gray",
    linewidth=0.3,
    legend=True,
    legend_kwds={
        "label": "Vulnerability Score\n(Poverty + Renter Share)",
        "orientation": "vertical",
    },
)
axes[0].set_title("Neighborhood Vulnerability Index", fontsize=13, fontweight="bold")
axes[0].axis("off")

# Plot 2: Request Rate per Capita
silent_map.plot(
    column="requests_per_1000",
    ax=axes[1],
    cmap="Greens",
    edgecolor="gray",
    linewidth=0.3,
    legend=True,
    legend_kwds={"label": "Requests per 1000 Residents", "orientation": "vertical"},
)

# Highlight silent neighborhoods with red outline
silent_only = silent_map[silent_map["is_silent"] == True]
if len(silent_only) > 0:
    silent_only.plot(
        ax=axes[1],
        facecolor="none",
        edgecolor="red",
        linewidth=2,
        label="Silent Neighborhoods",
    )

axes[1].set_title("311 Reporting Rate (per Capita)", fontsize=13, fontweight="bold")
axes[1].axis("off")

plt.tight_layout()
plt.savefig("../figures/silent_neighborhoods_map.png", dpi=300, bbox_inches="tight")
plt.show()

print("\n=== INTERPRETATION ===")
print("Left map: Darker red = more vulnerable (high poverty + renter share)")
print("Right map: Darker green = more 311 requests per capita")

print("RED OUTLINES: Silent neighborhoods (vulnerable but low reporting)")
print(f"\n✓ Figure saved to ../figures/silent_neighborhoods_map.png")

In [ ]:
# Scatter plot: Vulnerability vs. Request Rate
fig, ax = plt.subplots(figsize=(12, 8))

# Remove outliers using IQR method for requests_per_1000
Q1 = silent_df["requests_per_1000"].quantile(0.25)
Q3 = silent_df["requests_per_1000"].quantile(0.75)
IQR = Q3 - Q1
outlier_mask = (silent_df["requests_per_1000"] >= Q1 - 1.5 * IQR) & (
    silent_df["requests_per_1000"] <= Q3 + 1.5 * IQR
)
silent_df_clean = silent_df[outlier_mask]

print(
    f"Removed {len(silent_df) - len(silent_df_clean)} outliers (out of {len(silent_df)} tracts)"
)
print(
    f"Request rate range after outlier removal: {silent_df_clean['requests_per_1000'].min():.2f} to {silent_df_clean['requests_per_1000'].max():.2f}"
)

# Color by silent status
colors = silent_df_clean["is_silent"].map({True: "red", False: "steelblue"})
sizes = silent_df_clean["total_population"] / 10  # Scale point size by population

ax.scatter(
    silent_df_clean["vulnerability_score"],
    silent_df_clean["requests_per_1000"],
    c=colors,
    s=sizes,
    alpha=0.6,
    edgecolors="black",
    linewidth=0.5,
)

# Add reference lines
ax.axvline(
    vuln_75th,
    color="red",
    linestyle="--",
    linewidth=2,
    alpha=0.5,
    label=f"High Vulnerability (75th %ile)",
)
ax.axhline(
    req_25th,
    color="green",
    linestyle="--",
    linewidth=2,
    alpha=0.5,
    label=f"Low Request Rate (25th %ile)",
)

ax.set_xlabel("Vulnerability Score (normalized)", fontsize=12)
ax.set_ylabel("Requests per 1000 Residents", fontsize=12)
ax.set_title(
    "Silent Neighborhoods: High Vulnerability + Low Reporting\n(Outliers Removed)",
    fontsize=13,
    fontweight="bold",
)
ax.grid(True, alpha=0.3)

# Add legend
from matplotlib.patches import Patch

legend_elements = [
    Patch(facecolor="red", alpha=0.6, edgecolor="black", label="Silent Neighborhoods"),
    Patch(
        facecolor="steelblue", alpha=0.6, edgecolor="black", label="Other Neighborhoods"
    ),
]
ax.legend(handles=legend_elements, fontsize=11, loc="upper right")

plt.tight_layout()
plt.savefig("../figures/silent_neighborhoods_scatter.png", dpi=300, bbox_inches="tight")
plt.show()

print("\nPoint size represents total population.")

print("RED points = Silent neighborhoods (intervention priority)")
print(f"\n✓ Figure saved to ../figures/silent_neighborhoods_scatter.png")

### Silent Tracts Demographic Comparison

How do tracts with the lowest 311 reporting rates compare demographically to those with higher reporting?

We compare tracts in the bottom quartile of requests per capita to all others on key ACS indicators.


In [ ]:
# Step 1: Define silent tracts (bottom quartile of requests_per_capita)
from scipy.stats import ttest_ind

# Use requests_per_1000 as the reporting metric
requests_per_capita_q25 = silent_df["requests_per_1000"].quantile(0.25)

silent_df["is_silent_quartile"] = (
    silent_df["requests_per_1000"] <= requests_per_capita_q25
)

n_silent = silent_df["is_silent_quartile"].sum()
n_not_silent = (~silent_df["is_silent_quartile"]).sum()

print(f"=== SILENT TRACTS DEFINITION ===")
print(f"Silent tracts (bottom quartile): {n_silent}")
print(f"Non-silent tracts: {n_not_silent}")
print(
    f"Reporting rate threshold: {requests_per_capita_q25:.2f} requests per 1000 residents"
)

In [ ]:
# Step 2: Prepare comparison data with ACS characteristics
# Check for median_household_income; if not in silent_df, merge from sf_tracts_gdf
if "median_household_income" not in silent_df.columns:
    silent_df = silent_df.merge(
        sf_tracts_gdf[["GEOID", "median_household_income"]], on="GEOID", how="left"
    )

# Select demographic columns for comparison
comparison_cols = [
    "median_household_income",
    "pct_renter_occupied",
    "poverty_rate",
    "requests_per_1000",
]

# Ensure all columns exist
available_comparison_cols = [col for col in comparison_cols if col in silent_df.columns]

print(f"\nAvailable columns for comparison: {available_comparison_cols}")

# Create comparison groups
silent_tracts = silent_df[silent_df["is_silent_quartile"]]
non_silent_tracts = silent_df[~silent_df["is_silent_quartile"]]

print(
    f"\nSilent tracts n={len(silent_tracts)}, Non-silent tracts n={len(non_silent_tracts)}"
)

In [ ]:
# Step 3: Compute summary statistics and mean differences
comparison_results = []

for col in available_comparison_cols:
    # Get clean data (drop NaNs)
    silent_data = silent_tracts[col].dropna()
    non_silent_data = non_silent_tracts[col].dropna()

    # Compute means
    mean_silent = silent_data.mean()
    mean_non_silent = non_silent_data.mean()

    # Compute difference
    difference = mean_silent - mean_non_silent
    pct_difference = (
        (difference / mean_non_silent * 100) if mean_non_silent != 0 else np.nan
    )

    # Run t-test if column is income or renter share (as requested)
    if col in ["median_household_income", "pct_renter_occupied"]:
        t_stat, p_value = ttest_ind(silent_data, non_silent_data)
    else:
        t_stat, p_value = np.nan, np.nan

    comparison_results.append(
        {
            "Characteristic": col,
            "Silent Tracts (Mean)": mean_silent,
            "Non-Silent Tracts (Mean)": mean_non_silent,
            "Difference": difference,
            "% Difference": pct_difference,
            "t-statistic": t_stat,
            "p-value": p_value,
            "n_silent": len(silent_data),
            "n_non_silent": len(non_silent_data),
        }
    )

# Create summary table
summary_table = pd.DataFrame(comparison_results)

print("\n=== DEMOGRAPHIC COMPARISON: SILENT VS. NON-SILENT TRACTS ===\n")
display(summary_table)

In [ ]:
# Step 4: Detailed interpretation of t-test results
print("\n=== T-TEST RESULTS FOR KEY DEMOGRAPHIC DIFFERENCES ===\n")

for idx, row in summary_table.iterrows():
    if row["Characteristic"] in ["median_household_income", "pct_renter_occupied"]:
        print(f"{row['Characteristic']}:")

        if row["Characteristic"] == "median_household_income":
            print(f"  Silent tracts mean: ${row['Silent Tracts (Mean)']:,.0f}")
            print(f"  Non-silent tracts mean: ${row['Non-Silent Tracts (Mean)']:,.0f}")
            print(f"  Difference: ${row['Difference']:,.0f}")
        else:  # pct_renter_occupied
            print(f"  Silent tracts mean: {row['Silent Tracts (Mean)']*100:.1f}%")
            print(
                f"  Non-silent tracts mean: {row['Non-Silent Tracts (Mean)']*100:.1f}%"
            )
            print(f"  Difference: {row['Difference']*100:.1f} percentage points")

        print(f"  % Difference: {row['% Difference']:.1f}%")

        # Interpret p-value
        if row["p-value"] < 0.001:
            sig_level = "*** (p < 0.001 - highly significant)"
        elif row["p-value"] < 0.01:
            sig_level = "** (p < 0.01 - very significant)"
        elif row["p-value"] < 0.05:
            sig_level = "* (p < 0.05 - significant)"
        else:
            sig_level = "ns (not significant at p < 0.05)"

        print(
            f"  t-test: t = {row['t-statistic']:.3f}, p = {row['p-value']:.4f} {sig_level}"
        )
        print()

print("\nInterpretation:")
print("- Silent tracts have HIGHER median household income (but not significantly)")
print("- Silent tracts have SIGNIFICANTLY LOWER renter occupancy (35% lower)")
print(
    "- This suggests silent tracts are more owner-occupied, higher-income neighborhoods"
)

In [ ]:
# Visualization: Compare demographics between silent and non-silent tracts
fig, axes = plt.subplots(2, 2, figsize=(14, 10))
fig.suptitle(
    "Silent Tracts vs. Non-Silent Tracts: Demographic Comparison",
    fontsize=14,
    fontweight="bold",
)

# Plot 1: Median Household Income
ax = axes[0, 0]
data_income = [
    silent_tracts["median_household_income"].dropna(),
    non_silent_tracts["median_household_income"].dropna(),
]
bp = ax.boxplot(
    data_income, labels=["Silent\nTracts", "Non-Silent\nTracts"], patch_artist=True
)
for patch, color in zip(bp["boxes"], ["lightcoral", "lightblue"]):
    patch.set_facecolor(color)
ax.set_ylabel("Median Household Income ($)", fontsize=11)
ax.set_title("Income Distribution", fontsize=12, fontweight="bold")
ax.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f"${x/1000:.0f}K"))
ax.grid(True, alpha=0.3, axis="y")

# Plot 2: Renter Occupancy
ax = axes[0, 1]
data_renter = [
    silent_tracts["pct_renter_occupied"].dropna() * 100,
    non_silent_tracts["pct_renter_occupied"].dropna() * 100,
]
bp = ax.boxplot(
    data_renter, labels=["Silent\nTracts", "Non-Silent\nTracts"], patch_artist=True
)
for patch, color in zip(bp["boxes"], ["lightcoral", "lightblue"]):
    patch.set_facecolor(color)
ax.set_ylabel("Renter Occupied (%)", fontsize=11)
ax.set_title("Renter Occupancy Distribution", fontsize=12, fontweight="bold")
ax.grid(True, alpha=0.3, axis="y")

# Plot 3: Poverty Rate
ax = axes[1, 0]
data_poverty = [
    silent_tracts["poverty_rate"].dropna() * 100,
    non_silent_tracts["poverty_rate"].dropna() * 100,
]
bp = ax.boxplot(
    data_poverty, labels=["Silent\nTracts", "Non-Silent\nTracts"], patch_artist=True
)
for patch, color in zip(bp["boxes"], ["lightcoral", "lightblue"]):
    patch.set_facecolor(color)
ax.set_ylabel("Poverty Rate (%)", fontsize=11)
ax.set_title("Poverty Rate Distribution", fontsize=12, fontweight="bold")
ax.grid(True, alpha=0.3, axis="y")

# Plot 4: Reporting Rate
ax = axes[1, 1]
data_requests = [
    silent_tracts["requests_per_1000"].dropna(),
    non_silent_tracts["requests_per_1000"].dropna(),
]
bp = ax.boxplot(
    data_requests, labels=["Silent\nTracts", "Non-Silent\nTracts"], patch_artist=True
)
for patch, color in zip(bp["boxes"], ["lightcoral", "lightblue"]):
    patch.set_facecolor(color)
ax.set_ylabel("Requests per 1000 Residents", fontsize=11)
ax.set_title("311 Reporting Rate Distribution", fontsize=12, fontweight="bold")
ax.grid(True, alpha=0.3, axis="y")

plt.tight_layout()
plt.savefig(
    "../figures/silent_tracts_demographic_comparison.png", dpi=300, bbox_inches="tight"
)
plt.show()
print("\n✓ Figure saved to ../figures/silent_tracts_demographic_comparison.png")

## Service-Level Equity Analysis: Delay Correlations with Demographics

For which services does government responsiveness vary most by neighborhood income and housing tenure?

We analyze each service category separately to identify which service types show the strongest correlation between delays and demographic factors.


In [ ]:
# Step 1: Prepare data with service_base, delay, GEOID, and demographics
# Assume closed_req_mapped has service_base, delay_hours, and demographics are available
# If using service_col = "request_type", adjust to "service_base" if available

# Check if service_base column exists
service_analysis_col = (
    "service_base" if "service_base" in closed_req_mapped.columns else service_col
)

print(f"Using service column: {service_analysis_col}")

# Create working dataframe with necessary columns
required_cols = [
    service_analysis_col,
    "delay_hours",
    "GEOID",
    "median_household_income",
    "pct_renter_occupied",
]

# First, merge tract demographics with request data
req_with_demographics = req_with_tract[
    [service_analysis_col, "delay_hours", geoid_col_sjoin]
].copy()
req_with_demographics = req_with_demographics.rename(columns={geoid_col_sjoin: "GEOID"})

# Add demographic variables from sf_tracts_gdf
demo_vars = req_with_demographics.merge(
    sf_tracts_gdf[["GEOID", "median_household_income", "pct_renter_occupied"]],
    on="GEOID",
    how="left",
)

# Drop rows missing any required values
demo_vars_clean = demo_vars.dropna(
    subset=[
        service_analysis_col,
        "delay_hours",
        "median_household_income",
        "pct_renter_occupied",
    ]
)

print(f"\nRequests with complete data: {len(demo_vars_clean)}")
print(f"Unique services: {demo_vars_clean[service_analysis_col].nunique()}")

In [ ]:
# Step 2: Filter services with at least 100 resolved requests citywide
service_counts = demo_vars_clean[service_analysis_col].value_counts()
services_min_100 = service_counts[service_counts >= 100].index.tolist()

demo_vars_filtered = demo_vars_clean[
    demo_vars_clean[service_analysis_col].isin(services_min_100)
].copy()

print(f"Services with ≥100 requests: {len(services_min_100)}")
print(f"Total requests for these services: {len(demo_vars_filtered)}")
print(f"\nService request counts (top 10):")
print(demo_vars_filtered[service_analysis_col].value_counts().head(10))

In [ ]:
# Step 3: For each service, compute correlations with delay_hours
# Compute Pearson correlation between delay_hours and each demographic variable

service_correlations = []

for service in services_min_100:
    service_data = demo_vars_filtered[
        demo_vars_filtered[service_analysis_col] == service
    ].copy()

    n_requests = len(service_data)

    # Correlation with median_household_income
    income_delay = service_data[["median_household_income", "delay_hours"]].dropna()
    if len(income_delay) >= 5:  # Require minimum sample size
        r_income, p_income = pearsonr(
            income_delay["median_household_income"], income_delay["delay_hours"]
        )
    else:
        r_income, p_income = np.nan, np.nan

    # Correlation with percent_renter_occupied
    renter_delay = service_data[["pct_renter_occupied", "delay_hours"]].dropna()
    if len(renter_delay) >= 5:
        r_renter, p_renter = pearsonr(
            renter_delay["pct_renter_occupied"], renter_delay["delay_hours"]
        )
    else:
        r_renter, p_renter = np.nan, np.nan

    service_correlations.append(
        {
            "service": service,
            "n_requests": n_requests,
            "r_income_delay": r_income,
            "p_income_delay": p_income,
            "r_renter_delay": r_renter,
            "p_renter_delay": p_renter,
        }
    )

# Convert to tidy DataFrame
service_corr_df = pd.DataFrame(service_correlations).sort_values(
    "r_income_delay", ascending=False
)

print("=== SERVICE-LEVEL DELAY CORRELATIONS WITH DEMOGRAPHICS ===\n")
display(service_corr_df)

In [ ]:
# Interpretation: Highlight significant correlations
print("\n=== INTERPRETATION ===\n")

print("Services with STRONGEST POSITIVE income–delay correlation")
print("(higher income → longer delays):\n")
positive_income = service_corr_df[service_corr_df["r_income_delay"] > 0.2].head(5)
for idx, row in positive_income.iterrows():
    sig = (
        "***"
        if row["p_income_delay"] < 0.001
        else (
            "**"
            if row["p_income_delay"] < 0.01
            else "*" if row["p_income_delay"] < 0.05 else "ns"
        )
    )
    print(
        f"  {row['service']:40s}: r = {row['r_income_delay']:6.3f} ({sig}), n = {row['n_requests']}"
    )

print("\n\nServices with STRONGEST NEGATIVE income–delay correlation")
print("(higher income → shorter delays):\n")
negative_income = service_corr_df[service_corr_df["r_income_delay"] < -0.2].head(5)
for idx, row in negative_income.iterrows():
    sig = (
        "***"
        if row["p_income_delay"] < 0.001
        else (
            "**"
            if row["p_income_delay"] < 0.01
            else "*" if row["p_income_delay"] < 0.05 else "ns"
        )
    )
    print(
        f"  {row['service']:40s}: r = {row['r_income_delay']:6.3f} ({sig}), n = {row['n_requests']}"
    )

print("\n\nServices with STRONGEST renter–delay correlation")
print("(higher renter share → longer delays):\n")
renter_corr = service_corr_df.sort_values("r_renter_delay", ascending=False).head(5)
for idx, row in renter_corr.iterrows():
    sig = (
        "***"
        if row["p_renter_delay"] < 0.001
        else (
            "**"
            if row["p_renter_delay"] < 0.01
            else "*" if row["p_renter_delay"] < 0.05 else "ns"
        )
    )
    print(
        f"  {row['service']:40s}: r = {row['r_renter_delay']:6.3f} ({sig}), n = {row['n_requests']}"
    )

print("\n\nSignificance levels: *** p<0.001, ** p<0.01, * p<0.05, ns = not significant")

In [ ]:
# Visualization: Bar plot of income–delay correlations sorted by strength
fig, axes = plt.subplots(1, 2, figsize=(16, 10))

# Plot 1: Income-Delay Correlations
service_corr_sorted = service_corr_df.sort_values("r_income_delay")
colors_income = [
    "red" if x < -0.1 else "orange" if x < 0 else "lightblue" if x < 0.1 else "blue"
    for x in service_corr_sorted["r_income_delay"]
]

axes[0].barh(
    range(len(service_corr_sorted)),
    service_corr_sorted["r_income_delay"],
    color=colors_income,
)
axes[0].set_yticks(range(len(service_corr_sorted)))
axes[0].set_yticklabels(service_corr_sorted["service"], fontsize=8)
axes[0].set_xlabel("Correlation (delay hours vs. median income)", fontsize=11)
axes[0].set_title(
    "Service-Level Income–Delay Correlation\n(Red = inequitable: higher income → longer delays)",
    fontsize=12,
    fontweight="bold",
)
axes[0].axvline(0, color="black", linestyle="-", linewidth=0.8)
axes[0].grid(True, alpha=0.3, axis="x")

# Plot 2: Renter–Delay Correlations
service_corr_renter = service_corr_df.sort_values("r_renter_delay")
colors_renter = [
    "red" if x < -0.1 else "orange" if x < 0 else "lightblue" if x < 0.1 else "blue"
    for x in service_corr_renter["r_renter_delay"]
]

axes[1].barh(
    range(len(service_corr_renter)),
    service_corr_renter["r_renter_delay"],
    color=colors_renter,
)
axes[1].set_yticks(range(len(service_corr_renter)))
axes[1].set_yticklabels(service_corr_renter["service"], fontsize=8)
axes[1].set_xlabel("Correlation (delay hours vs. renter share)", fontsize=11)
axes[1].set_title(
    "Service-Level Renter–Delay Correlation\n(Red = inequitable: higher renter share → longer delays)",
    fontsize=12,
    fontweight="bold",
)
axes[1].axvline(0, color="black", linestyle="-", linewidth=0.8)
axes[1].grid(True, alpha=0.3, axis="x")

plt.tight_layout()
plt.savefig("../figures/service_level_correlations.png", dpi=300, bbox_inches="tight")
plt.show()

print(
    "\nColor coding: Red = negative correlation (inequitable), Blue = positive correlation"
)
print(f"✓ Figure saved to ../figures/service_level_correlations.png")

## Counterfactual / Residual Equity Analysis

**Question**: After controlling for service type, request volume, and demographic factors, which neighborhoods still experience systematically longer delays?

We fit a regression model predicting delay_hours using:

- Service category (fixed effects)
- Median household income
- Percent renter-occupied
- Request volume per tract

Then identify tracts with the largest **positive residuals** (actual delays exceed what the model predicts).


In [ ]:
# Step 1: Prepare request-level DataFrame with all required variables
# We need: delay_hours, service_base, GEOID, median_household_income, pct_renter_occupied, request_volume_tract

# Start with demo_vars_clean which already has delay_hours, service, GEOID, and demographics
residual_df = demo_vars_clean.copy()

# Use service_analysis_col as service_base
residual_df = residual_df.rename(columns={service_analysis_col: "service_base"})

# Compute request volume per tract
request_volume_by_tract = (
    residual_df.groupby("GEOID").size().reset_index(name="request_volume_tract")
)

# Merge request volume back to request-level data
residual_df = residual_df.merge(request_volume_by_tract, on="GEOID", how="left")

print("=== STEP 1: REQUEST-LEVEL DATA PREPARATION ===")
print(f"Total requests with complete data: {len(residual_df)}")
print(f"Unique tracts: {residual_df['GEOID'].nunique()}")
print(f"Unique service types: {residual_df['service_base'].nunique()}")
print("\nColumns available:")
print(
    residual_df[
        [
            "delay_hours",
            "service_base",
            "GEOID",
            "median_household_income",
            "pct_renter_occupied",
            "request_volume_tract",
        ]
    ].head()
)

In [ ]:
# Step 2: Fit linear regression model using statsmodels
# Predict delay_hours using service_base (categorical), income, renter_pct, and request_volume

import statsmodels.api as sm
from statsmodels.formula.api import ols

# Prepare data: ensure no missing values in predictors
model_data = residual_df[
    [
        "delay_hours",
        "service_base",
        "median_household_income",
        "pct_renter_occupied",
        "request_volume_tract",
        "GEOID",
    ]
].dropna()

print("=== STEP 2: FIT LINEAR REGRESSION MODEL ===")
print(f"Data for modeling: {len(model_data)} requests")
print(f"\nModel specification:")
print(
    "  delay_hours ~ C(service_base) + median_household_income + pct_renter_occupied + request_volume_tract"
)
print("\nC(service_base) creates categorical fixed effects for each service type")

# Fit OLS regression with service_base as categorical predictor
# C() treats service_base as categorical and creates dummy variables
model = ols(
    "delay_hours ~ C(service_base) + median_household_income + pct_renter_occupied + request_volume_tract",
    data=model_data,
).fit()

print("\n=== MODEL SUMMARY ===")
print(model.summary())

print(f"\nModel R-squared: {model.rsquared:.4f}")
print(f"Model Adjusted R-squared: {model.rsquared_adj:.4f}")
print(f"Number of observations: {model.nobs:.0f}")

In [ ]:
# Step 3: Compute residuals for each request
# Residual = Actual delay - Predicted delay
# Positive residuals = slower than model predicts (after controlling for all factors)

model_data["predicted_delay"] = model.fittedvalues
model_data["residual"] = model.resid

print("=== STEP 3: COMPUTE RESIDUALS ===")
print(f"Residuals computed for {len(model_data)} requests")
print("\nResidual statistics:")
print(model_data["residual"].describe())

print("\n=== INTERPRETATION ===")
print("Positive residual: Actual delay LONGER than model predicts (inequitable)")
print("Negative residual: Actual delay SHORTER than model predicts (advantaged)")
print("\nExamples:")
display(
    model_data[
        ["GEOID", "service_base", "delay_hours", "predicted_delay", "residual"]
    ].head(10)
)

In [ ]:
# Step 4: Aggregate residuals to tract level
# Compute mean, median, and count of residuals per tract

tract_residuals = (
    model_data.groupby("GEOID")
    .agg({"residual": ["mean", "median", "count"]})
    .reset_index()
)

# Flatten column names
tract_residuals.columns = ["GEOID", "mean_residual", "median_residual", "n_requests"]

print("=== STEP 4: AGGREGATE RESIDUALS TO TRACT LEVEL ===")
print(f"Total tracts: {len(tract_residuals)}")
print("\nTract-level residual statistics:")
print(tract_residuals[["mean_residual", "median_residual", "n_requests"]].describe())

# Add demographic info for context
tract_residuals = tract_residuals.merge(
    sf_tracts_gdf[
        ["GEOID", "median_household_income", "pct_renter_occupied", "poverty_rate"]
    ],
    on="GEOID",
    how="left",
)

In [ ]:
# Step 5: Filter to tracts with at least 30 requests and identify top 15 worst performers
# These are tracts with the highest mean residuals (systematically slower than expected)

min_requests = 30
tract_residuals_filtered = tract_residuals[
    tract_residuals["n_requests"] >= min_requests
].copy()

print(f"=== STEP 5: FILTER & IDENTIFY TOP UNDERPERFORMING TRACTS ===")
print(
    f"Tracts with at least {min_requests} requests: {len(tract_residuals_filtered)} / {len(tract_residuals)}"
)

# Sort by mean residual (highest = worst)
tract_residuals_sorted = tract_residuals_filtered.sort_values(
    "mean_residual", ascending=False
)

# Get top 15
top_15_worst = tract_residuals_sorted.head(15)

print(f"\n=== TOP 15 TRACTS WITH HIGHEST UNEXPLAINED DELAYS ===")
print(
    "(Systematically slower than model predicts, even after controlling for service type, income, renter share, and volume)"
)
print()
display(
    top_15_worst[
        [
            "GEOID",
            "mean_residual",
            "median_residual",
            "n_requests",
            "median_household_income",
            "pct_renter_occupied",
            "poverty_rate",
        ]
    ]
)

print("\n=== KEY INSIGHTS ===")
print(
    f"Mean residual range for top 15: {top_15_worst['mean_residual'].min():.2f} to {top_15_worst['mean_residual'].max():.2f} hours"
)
print(
    f"Average income of top 15: ${top_15_worst['median_household_income'].mean():,.0f}"
)
print(
    f"Average renter share of top 15: {top_15_worst['pct_renter_occupied'].mean()*100:.1f}%"
)
print(f"Average poverty rate of top 15: {top_15_worst['poverty_rate'].mean()*100:.1f}%")

In [ ]:
# Visualization: Choropleth map of mean residual delay by census tract
# Using diverging color scale centered at 0, clipped at 5th and 95th percentiles

# Merge residuals with tract geometries
residual_map = sf_tracts_gdf.merge(tract_residuals_filtered, on="GEOID", how="left")

fig, ax = plt.subplots(1, 1, figsize=(14, 12))

# Clip extreme values at 5th and 95th percentiles for better visualization
vmin_res = tract_residuals_filtered["mean_residual"].quantile(0.05)
vmax_res = tract_residuals_filtered["mean_residual"].quantile(0.95)

# Ensure color scale is centered at 0 by making vmin and vmax symmetric
max_abs = max(abs(vmin_res), abs(vmax_res))
vmin_centered = -max_abs
vmax_centered = max_abs

# Create choropleth with diverging colormap centered at 0
residual_map.plot(
    column="mean_residual",
    ax=ax,
    cmap="RdBu_r",  # Red = positive (slower), Blue = negative (faster)
    edgecolor="gray",
    linewidth=0.3,
    legend=True,
    vmin=vmin_centered,
    vmax=vmax_centered,
    legend_kwds={
        "label": "Residual Delay (Hours)",
        "orientation": "horizontal",
        "shrink": 0.6,
    },
)

ax.set_title(
    "Residual Service Delays by Census Tract\n(After Controlling for Service Type, Demographics, and Request Volume)",
    fontsize=14,
    fontweight="bold",
)
ax.axis("off")
plt.tight_layout()
plt.savefig("../figures/residual_delays_choropleth.png", dpi=300, bbox_inches="tight")
plt.show()

print("\n=== MAP INTERPRETATION ===")
print(
    f"Color scale clipped at 5th percentile ({vmin_res:.2f} hrs) and 95th percentile ({vmax_res:.2f} hrs)"
)
print(
    f"Symmetric range: {vmin_centered:.2f} to {vmax_centered:.2f} hours (centered at 0)"
)
print("\nRed areas = Positive residuals (systematically SLOWER than model predicts)")
print("Blue areas = Negative residuals (systematically FASTER than model predicts)")
print("White/light areas = Near-zero residuals (delays match model predictions)")
print("\nThese residuals control for:")
print("  • Service type (categorical fixed effects)")
print("  • Median household income")
print("  • Percent renter-occupied housing")
print("  • Total request volume per tract")
print(f"\n✓ Figure saved to ../figures/residual_delays_choropleth.png")

### Robustness Check: Median Residuals with Spearman Correlations

To ensure our findings are robust to outliers and non-linear relationships, we repeat the analysis using:

- **Median residuals** instead of mean (more resistant to extreme values)
- **Spearman rank correlations** instead of Pearson (captures monotonic relationships without assuming linearity)

We then compare which tracts appear as underperforming under both approaches.


In [ ]:
# Step 1: Identify top 15 tracts by MEDIAN residual (instead of mean)
# Sort by median residual to identify worst performers using a robust metric

tract_residuals_sorted_median = tract_residuals_filtered.sort_values(
    "median_residual", ascending=False
)

# Get top 15 by median residual
top_15_worst_median = tract_residuals_sorted_median.head(15)

print("=== TOP 15 TRACTS BY MEDIAN RESIDUAL (Robust to Outliers) ===")
print(
    "(Tracts with highest median unexplained delays, more resistant to extreme values)\n"
)
display(
    top_15_worst_median[
        [
            "GEOID",
            "median_residual",
            "mean_residual",
            "n_requests",
            "median_household_income",
            "pct_renter_occupied",
            "poverty_rate",
        ]
    ]
)

print("\n=== COMPARISON: MEAN vs MEDIAN RESIDUAL ===")
print(
    f"Median residual range for top 15: {top_15_worst_median['median_residual'].min():.2f} to {top_15_worst_median['median_residual'].max():.2f} hours"
)
print(
    f"Mean residual range for top 15: {top_15_worst_median['mean_residual'].min():.2f} to {top_15_worst_median['mean_residual'].max():.2f} hours"
)
print(
    f"\nAverage income of top 15 (median method): ${top_15_worst_median['median_household_income'].mean():,.0f}"
)
print(
    f"Average renter share of top 15 (median method): {top_15_worst_median['pct_renter_occupied'].mean()*100:.1f}%"
)
print(
    f"Average poverty rate of top 15 (median method): {top_15_worst_median['poverty_rate'].mean()*100:.1f}%"
)

In [ ]:
# Step 2: Overlap analysis - Which tracts appear in BOTH top 15 lists?
# Compare mean-based and median-based identification of worst performers

top_15_mean_geoids = set(top_15_worst["GEOID"])
top_15_median_geoids = set(top_15_worst_median["GEOID"])

# Find overlap
overlap_geoids = top_15_mean_geoids.intersection(top_15_median_geoids)
mean_only = top_15_mean_geoids - top_15_median_geoids
median_only = top_15_median_geoids - top_15_mean_geoids

print("=== TRACT OVERLAP ANALYSIS ===\n")
print(f"Tracts in BOTH top 15 lists: {len(overlap_geoids)}")
print(f"Tracts only in mean top 15: {len(mean_only)}")
print(f"Tracts only in median top 15: {len(median_only)}")
print(f"\nOverlap rate: {len(overlap_geoids)/15*100:.1f}%")

if len(overlap_geoids) > 0:
    print(f"\n=== CONSISTENT UNDERPERFORMERS (in both lists) ===")
    overlap_df = tract_residuals_filtered[
        tract_residuals_filtered["GEOID"].isin(overlap_geoids)
    ].sort_values("mean_residual", ascending=False)
    display(
        overlap_df[
            [
                "GEOID",
                "mean_residual",
                "median_residual",
                "n_requests",
                "median_household_income",
                "pct_renter_occupied",
            ]
        ]
    )

if len(mean_only) > 0:
    print(f"\n=== TRACTS ONLY IN MEAN TOP 15 (potentially outlier-driven) ===")
    mean_only_df = tract_residuals_filtered[
        tract_residuals_filtered["GEOID"].isin(mean_only)
    ].sort_values("mean_residual", ascending=False)
    display(
        mean_only_df[
            [
                "GEOID",
                "mean_residual",
                "median_residual",
                "n_requests",
            ]
        ]
    )

if len(median_only) > 0:
    print(f"\n=== TRACTS ONLY IN MEDIAN TOP 15 (more consistent delays) ===")
    median_only_df = tract_residuals_filtered[
        tract_residuals_filtered["GEOID"].isin(median_only)
    ].sort_values("median_residual", ascending=False)
    display(
        median_only_df[
            [
                "GEOID",
                "mean_residual",
                "median_residual",
                "n_requests",
            ]
        ]
    )

In [ ]:
# Step 3: Re-run regression with Spearman correlations approach
# Use rank-based method that doesn't assume linear relationships

from scipy.stats import spearmanr

# For Spearman, we compute correlations at the tract level (not request level)
# between median residuals and demographic variables

print("=== SPEARMAN RANK CORRELATIONS: MEDIAN RESIDUAL vs. DEMOGRAPHICS ===")
print("(Non-parametric test for monotonic relationships)\n")

# Variables to test
demo_vars_to_test = [
    "median_household_income",
    "pct_renter_occupied",
    "poverty_rate",
    "n_requests",
]

spearman_results = []

for var in demo_vars_to_test:
    # Get clean data (both variables present)
    clean_data = tract_residuals_filtered[[var, "median_residual"]].dropna()

    if len(clean_data) >= 5:
        rho, p_value = spearmanr(clean_data[var], clean_data["median_residual"])

        # Interpret significance
        if p_value < 0.001:
            sig = "***"
        elif p_value < 0.01:
            sig = "**"
        elif p_value < 0.05:
            sig = "*"
        else:
            sig = "ns"

        spearman_results.append(
            {
                "Variable": var,
                "Spearman_rho": rho,
                "p_value": p_value,
                "significance": sig,
                "n_tracts": len(clean_data),
            }
        )

        print(
            f"{var:30s}: ρ = {rho:7.3f}, p = {p_value:.4f} {sig} (n={len(clean_data)})"
        )

spearman_df = pd.DataFrame(spearman_results)

print("\n\nSignificance levels: *** p<0.001, ** p<0.01, * p<0.05, ns = not significant")
print("\n=== INTERPRETATION ===")
print(
    "Positive ρ: Higher values of demographic variable → higher median residuals (slower)"
)
print(
    "Negative ρ: Higher values of demographic variable → lower median residuals (faster)"
)
print("\nSpearman correlation captures ANY monotonic relationship (not just linear)")

In [ ]:
# Step 4: Summary comparison table - Mean/Pearson vs Median/Spearman

print("=== COMPREHENSIVE COMPARISON: MEAN/PEARSON vs MEDIAN/SPEARMAN ===\n")

# Create comparison summary
comparison_summary = {
    "Metric": [
        "Aggregation Method",
        "Correlation Type",
        "Resistance to Outliers",
        "Assumes Linearity",
        "Top 15 Tracts Identified",
        "Overlap with Other Method",
        "Avg Income of Top 15",
        "Avg Renter % of Top 15",
        "Avg Poverty % of Top 15",
    ],
    "Mean + Pearson\n(Original)": [
        "Mean residual",
        "Pearson (linear)",
        "Low (sensitive)",
        "Yes",
        f"{len(top_15_mean_geoids)} tracts",
        f"{len(overlap_geoids)} ({len(overlap_geoids)/15*100:.0f}%)",
        f"${top_15_worst['median_household_income'].mean():,.0f}",
        f"{top_15_worst['pct_renter_occupied'].mean()*100:.1f}%",
        f"{top_15_worst['poverty_rate'].mean()*100:.1f}%",
    ],
    "Median + Spearman\n(Robust)": [
        "Median residual",
        "Spearman (rank)",
        "High (robust)",
        "No (monotonic only)",
        f"{len(top_15_median_geoids)} tracts",
        f"{len(overlap_geoids)} ({len(overlap_geoids)/15*100:.0f}%)",
        f"${top_15_worst_median['median_household_income'].mean():,.0f}",
        f"{top_15_worst_median['pct_renter_occupied'].mean()*100:.1f}%",
        f"{top_15_worst_median['poverty_rate'].mean()*100:.1f}%",
    ],
}

comparison_df = pd.DataFrame(comparison_summary)
display(comparison_df)

print("\n=== KEY FINDINGS ===\n")

if len(overlap_geoids) >= 10:
    print(
        f"✓ STRONG AGREEMENT ({len(overlap_geoids)}/15 tracts overlap): Results are ROBUST"
    )
    print("  → These tracts consistently show unexplained delays regardless of method")
    print("  → High confidence in policy intervention targeting")
elif len(overlap_geoids) >= 5:
    print(
        f"◐ MODERATE AGREEMENT ({len(overlap_geoids)}/15 tracts overlap): Some sensitivity"
    )
    print("  → Core set of underperforming tracts identified")
    print("  → Additional tracts may be influenced by outliers or non-linearity")
else:
    print(
        f"✗ LOW AGREEMENT ({len(overlap_geoids)}/15 tracts overlap): Results are SENSITIVE"
    )
    print("  → Findings vary significantly by method")
    print("  → Recommend using median/Spearman for more robust identification")

print(
    f"\nDemographic comparison: {'Similar' if abs(top_15_worst['median_household_income'].mean() - top_15_worst_median['median_household_income'].mean()) < 10000 else 'Different'} income profiles"
)
print(
    f"Renter share comparison: {'Similar' if abs(top_15_worst['pct_renter_occupied'].mean() - top_15_worst_median['pct_renter_occupied'].mean()) < 0.1 else 'Different'} housing tenure"
)